# GEP
https://arxiv.org/pdf/cs/0102027

## Components of GEP

- *Chromosome*: Linear string of symbols (genes) representing *functions* and *terminals* (e.g. a Single-Gene Chromosome: *+ a b \* c d*, where *+* is the root, *a* and *\** are children of *+*, while *b* and *c* are children of *\**)
<br> *Genes* are fixed-length strings, divided into two parts: *Heads* consists of *functions* and *terminals*, whereas *tails* contains only *terminals*. Essentially, the *head* generates any kind of *expression tree*, and the *tail* ensures that they have enough arguments to form a valid tree.  The number of genes is decided.
<br> The length of the *head* $h$ is chosen, and the length of the *tail* $t$ depends on it. $t = h(n-1)+1$, where $n$ is the maximum number of arguments for the function with most arguments (e.g. $n = 2$ for *+*).
<br> By default the linking function of different genes within a chromosome is chosen in advance, but the linking operation could also be introduced in the chromosome. 
- *Functions*: Operators s.a. +, -, *, /, or custom functions
- *Terminals*: Variables/Factors
- *Expression Tree (ET)*: Decoded form of the chromosome, used for evaluation (tree-like structure (phenotype)). Does not need to have the same size as the Chromosome.
- *Fitness Function*: Function to evaluate goodness of solution

## Steps
1. Create Initial Population: Random chromosomes
2. Decode Chromosomes to ETs
3. Evaluate Fítness of each individual
4. Decide whether to continue or stop (Termination Condition)
5. Select Surviving Individuals:
    - Elitism, to ensure that the top X individuals are part of the next generation (skipping the genetic operations) (and replicate them to the next generation).
    - Remaining population is subject to other selection procedures (e.g. roulette wheel) and genetic operations.
6. Apply Genetic Operations
    - Except for mutation, a chromosome can only be modified once per operator.
    - Mutation: Randomly change part of chromosome (each position in the chromosome has a mutation rate)
    <br> In the paper, they state that the typical *mutation rate* $p_m$ is two point mutations per chromosome $p_m = 2/positions\_in\_chromosome$. Every position has a $p_m$ chance to be mutated.
    - Transpositions: 
        - IS Transposition: Copy and insert randomly selected segment of chromosome into the head of the same gene, at a random position other than the very start. The head of the gene is trimmed to maintain the fixed size.
    <br> Typically, an IS transposition rate $p_{is}$ of 0.1 is used and a set of 3 different lengths (e.g. [1,2,3]) are used. The start position of the IS element, its length and insertion point are selected randomly.
        - RIS Transposition: Copy and insert randomly selected segment of chromosome starting with a function to the root (start) of the head.
    <br> Typically the RIS transposition rate $p_{ris}$ is 0.1 and there is a set of 3 different possible IS element lengths. A point is randomly chosen in the head, and the gene is scanned downstream until a function is found. The last symbols of the head are lost (as many as the IS length)
        - Gene Transposition: Randomly selected gene (other than first) is moved (not copied!) to the beginning of the chromosome. Depending on the linking function, this may not have any impact on the chromosomes fitness, but it has an affect in crossover, allowing the duplication of genes. Typically, $p_g$ is 0.1.
    - Recombination: Every pair (after selection mechanism) has a chance to produce 2 off-springs, or otherwise proceed to the next generation. The overall recombination rate proposed in the paper is 0.7.
        - One-point Recombination: Chooses random position as crossover point, and exchange material at this point forming two offsprings. The probability $p_{1r}$ for this is usually the highest.
        - Two-point Recombination: Chooses 2 random positions as crossover points and exchanges material at this point. Probability is $p_{2r}$.
        - Gene Recombination: Chooses a random gene position and exchanges this one among the parents. Probability $p_{gr}$ is usually lower than the other two; frequently set to 0.1.
7. Repeat through generations from step 4 onwards

In [34]:
import numpy as np
import math
import operator
import sympy as sp

In [132]:
class GEP:
    def __init__(self, num_genes, head_length, function_map, function_arity, terminal_set, terminal_data): # todo : pass probabilities?
        self.num_genes = num_genes
        self.head_length = head_length
        self.tail_length = head_length * (np.max(list(function_arity.values())) - 1) + 1
        self.function_map = function_map
        self.function_arity = function_arity
        self.chromosome_length = num_genes * (head_length + self.tail_length)
        self.symbols_head = np.array(list(function_map.keys()) + terminal_set)
        self.symbols_tail = np.array(terminal_set)
        self.population = None
        self.fitness_scores = np.array([])
        self.terminal_data = terminal_data
        # compute gene split points
        self.gene_indices = [(i * (head_length+self.tail_length), (i + 1) * (head_length+self.tail_length)) for i in range(self.num_genes)]
        

    def initialize_population(self, pop_size):
        # create random heads and tails for every gene
        heads = np.random.choice(self.symbols_head, size=(pop_size, self.num_genes, self.head_length))
        tails = np.random.choice(self.symbols_tail, size=(pop_size, self.num_genes, self.tail_length))
        # combine heads and tails for each gene and flatten along the gene axis
        self.population = np.concatenate((heads, tails), axis=2).reshape(pop_size, self.chromosome_length)
        return self.population

    def decode_chromosome_factors(self, chromosome):
        """
        Decode ALL genes of a SINGLE chromosome into a list of matrices.
        Returns: [gene_0_matrix, gene_1_matrix, ..., gene_(num_genes-1)_matrix]
        where each matrix is shape = terminal_data[someSymbol].shape.
        """
        gene_matrices = []
        for g_start, g_end in self.gene_indices:
            # Slice out the gene
            gene_slice = chromosome[g_start:g_end]
            # Decode gene
            gene_matrix = self._decode_gene_postfix_conversion(gene_slice)
            gene_matrices.append(gene_matrix)
        return gene_matrices
    
    def compute_population_fitness(self, population, fitness_fn, chunk_size=100):
        """
        Compute fitness for each chromosome in `population` by:
          1) Decoding each gene into a (rows, cols) matrix,
          2) Passing all gene matrices to `fitness_fn`,
          3) Storing the scalar fitness in an output array.
    
        We do this in chunks to avoid huge memory usage.
        """
        pop_size = population.shape[0]
        fitness_scores = np.empty(pop_size, dtype=np.float64)
    
        # Process population in small chunks
        for start in range(0, pop_size, chunk_size):
            end = min(start + chunk_size, pop_size)
    
            for i in range(start, end):
                chromosome = population[i]  # shape: (chromosome_length,)
    
                # 1) Decode each gene
                gene_matrices = []
                for g_idx, (g_start, g_end) in enumerate(self.gene_indices):
                    # gene is the slice
                    gene_slice = chromosome[g_start:g_end]
                    decoded_mat = self._decode_gene_postfix_conversion(gene_slice)
                    gene_matrices.append(decoded_mat)
    
                # 2) Compute fitness
                fitness = fitness_fn(gene_matrices)
                fitness_scores[i] = fitness

        return fitness_scores

    def _decode_gene_postfix_conversion(self, gene):
        """
        Convert a single gene (prefix) -> postfix, then evaluate to get a (rows, cols) matrix.
        """
        postfix_expr = self._prefix_to_postfix(gene)
        return self._evaluate_postfix(postfix_expr)

    def _prefix_to_postfix(self, gene):
        """
        Convert a prefix-encoded gene to postfix in a single pass.
        """
        idx = 0
        def convert():
            nonlocal idx
            symbol = gene[idx]
            idx += 1
            if symbol in self.function_arity:
                ar = self.function_arity[symbol]
                children = []
                for _ in range(ar):
                    children.extend(convert())
                children.append(symbol)
                return children
            else:
                return [symbol]
        return convert()

    def _evaluate_postfix(self, postfix_expr):
        """
        Evaluate a postfix expression. Each operator in self.function_map
        is expected to do in-place operations if possible.
        """
        stack = []
        for sym in postfix_expr:
            if sym in self.function_map:
                ar = self.function_arity[sym]
                # Pop arguments in reverse order so we apply them in the correct order.
                args = [stack.pop() for _ in range(ar)][::-1]
                result = self.function_map[sym](*args)
                stack.append(result)
            else:
                # Terminal symbol -> fetch the terminal_data matrix
                stack.append(self.terminal_data[sym])
        # By construction, exactly one item left
        return stack[0]
    
    # evaluates fitness of every chromosome in population using custom fitness function
    def _evaluate_population(self, pop, fitness_function):
        pass

# todo: function and terminal set
# todo: decode 
# todo: early stop ?

In [133]:
# Define the test parameters
num_genes = 6
head_length = 6
terminal_set = ['a', 'b', 'c']
rows, cols = 2000, 100  # Shape of the terminal matrices

# Randomly generate terminal data
terminal_data = {t: np.random.rand(rows, cols) for t in terminal_set}

# Define function map and arity
function_map = {
    '+': np.add,
    '-': np.subtract,
    '*': np.multiply,
    '/': lambda x, y: np.divide(x, y, where=y != 0)  # Safe division
}
function_arity = {op: 2 for op in function_map.keys()}

# Create a GEP instance
gep = GEP(
    num_genes=num_genes,
    head_length=head_length,
    function_map=function_map,
    function_arity=function_arity,
    terminal_set=terminal_set,
    terminal_data=terminal_data
)

# Initialize population
population = gep.initialize_population(pop_size=5000)

def my_fitness(gene_mats):
    combined = np.zeros_like(gene_mats[0])  # shape = (rows, cols)
    for mat in gene_mats:
        combined += mat
    return np.sum(combined)

# Compute fitness without storing giant 4D arrays
fitness_scores = gep.compute_population_fitness(population, my_fitness, chunk_size=100)
print("Fitness array shape:", fitness_scores.shape)  # (5000,)

Fitness array shape: (5000,)


In [121]:
# find max index of fitness_scores
max_idx = np.argmax(fitness_scores)
print(max_idx, fitness_scores[max_idx])
min_idx = np.argmin(fitness_scores)
print(min_idx, fitness_scores[min_idx])

# get the chromosome of the max index and min index
max_chromosome = population[max_idx]
min_chromosome = population[min_idx]

print(max_chromosome)
print(min_chromosome)

4224 2.0992877447329646e+24
998 -4.0421787702593423e+21
['b' '-' '/' '/' '/' '-' 'b' 'c' 'a' 'a' 'a' 'b' 'b' '/' 'b' '*' 'a' '*'
 '*' 'a' 'a' 'a' 'c' 'b' 'c' 'b' 'c' '+' '/' '-' '/' '-' 'c' 'c' 'c' 'c'
 'a' 'b' 'a' 'a' '/' 'a' 'a' '+' '/' 'c' 'b' 'c' 'a' 'c' 'c' 'a' '*' 'c'
 '/' '*' 'a' '/' 'b' 'a' 'b' 'c' 'c' 'c' 'b' 'c' 'a' 'b' 'b' '+' '*' 'b'
 'a' 'b' 'b' 'a' 'a' 'c']
['*' '/' 'a' 'c' '/' 'a' 'b' 'c' 'c' 'c' 'a' 'b' 'b' 'b' 'b' '-' '-' '*'
 '/' 'c' 'a' 'c' 'b' 'b' 'b' 'c' 'c' '-' '/' 'c' '-' '+' 'b' 'c' 'a' 'a'
 'c' 'a' 'a' '/' 'a' '-' '/' 'a' '/' 'a' 'c' 'c' 'c' 'a' 'b' 'c' '/' '-'
 'a' 'b' 'c' '-' 'c' 'a' 'b' 'a' 'c' 'a' 'a' '*' '/' '*' 'c' 'b' 'a' 'c'
 'c' 'c' 'c' 'c' 'a' 'c']
